In [1]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import TensorBoard
from tensorflow import keras
import tensorflow as tf
from scipy import stats
import pandas as pd
import datetime, os
import numpy as np

Using TensorFlow backend.


In [2]:
# Import the preprocessed file we made
print("File name: ") 
file_name = input()

dataFrame = pd.read_csv(r'data/p/' + file_name + '.csv')
dataFrame.head()

File name: 
all_NSP


,duration,AC,FM,UC,DL,DS,DP,DR,LB,AC.1,...,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency,NSP
0,117,0.000000,0.0,0.000000,0.000,0.0,0.0,0.0,0.259259,0.000000,...,0.110092,0.034483,0.111111,0.0,0.472441,0.587156,0.403670,0.271375,1.0,2
1,627,0.153846,0.0,0.173913,0.125,0.0,0.0,0.0,0.481481,0.315789,...,0.165138,0.655172,0.333333,0.1,0.637795,0.577982,0.577982,0.044610,0.5,1
2,602,0.076923,0.0,0.217391,0.125,0.0,0.0,0.0,0.500000,0.157895,...,0.165138,0.655172,0.277778,0.1,0.637795,0.568807,0.559633,0.048327,0.5,1
3,781,0.076923,0.0,0.260870,0.125,0.0,0.0,0.0,0.518519,0.157895,...,0.027523,0.413793,0.611111,0.0,0.606299,0.559633,0.550459,0.048327,1.0,1
4,614,0.153846,0.0,0.217391,0.000,0.0,0.0,0.0,0.481481,0.368421,...,0.027523,0.413793,0.500000,0.0,0.606299,0.577982,0.559633,0.040892,1.0,1


In [3]:
num_features = dataFrame.shape[1] - 1
print("We have " + str(num_features) + " features") 

We have 29 features


In [7]:
target_name = input()
target = dataFrame.pop(target_name)


dataset = tf.data.Dataset.from_tensor_slices((dataFrame.values, target.values))

for feat, targ in dataset.take(5):
  print('Features: {}, Target: {}'.format(feat, targ))

NSP
Features: [1.17000000e+02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.59259259e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 8.13333333e-01
 4.41176471e-02 4.72527473e-01 4.73372781e-02 3.44632768e-01
 1.10091743e-01 3.44827586e-02 1.11111111e-01 0.00000000e+00
 4.72440945e-01 5.87155963e-01 4.03669725e-01 2.71375465e-01
 1.00000000e+00], Target: 2
Features: [6.27000000e+02 1.53846154e-01 0.00000000e+00 1.73913043e-01
 1.25000000e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.81481481e-01 3.15789474e-01 0.00000000e+00 4.00000000e-01
 2.00000000e-01 0.00000000e+00 0.00000000e+00 6.66666667e-02
 2.79411765e-01 0.00000000e+00 2.05128205e-01 7.17514124e-01
 1.65137615e-01 6.55172414e-01 3.33333333e-01 1.00000000e-01
 6.37795276e-01 5.77981651e-01 5.77981651e-01 4.46096654e-02
 5.00000000e-01], Target: 1
Features: [6.02000000e+02 7.69230769e-02 0.00000000e+00 2.17391304

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

This model will be used for the NSP classification

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(128, input_dim=num_features, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Tensorboard stuff
log_dir = os.path.join(
    "logs",
    datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir, histogram_freq=1)

# Train the model
model.fit(x=X_train, 
          y=y_train, 
          epochs=100,
          batch_size = 50,
          shuffle=False,
          validation_data=(X_val, y_val),
          callbacks=[tensorboard_callback],
          verbose=1)